# Carregando bibliotecas

In [ ]:
!pip install netCDF4
!pip install cfgrib
!pip install cartopy
!pip install metpy

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from metpy.units import units
import matplotlib as mpl

# Importando dados

In [ ]:
#################################################################
# IMPORTANDO DADOS DO ERA5 --> Registros de vento, pressão atmosférica, temperatura do ar e temperatura do ponto de orvalho
#################################################################
ds_ERA5 = xr.open_dataset('/content/ERA5_2015-2024.grib', engine='cfgrib')
print(ds_ERA5)

In [ ]:
###################################################
# IMPORTANDO DADOS DO MERGE-CPTEC --> Registros de Chuva
###################################################
ds_MERGE = xr.open_dataset('/content/MERGE_CPTEC_BRASIL_2015-2024.nc')
print(ds_MERGE)

In [ ]:
# Calculando a Velocidade do Vento a 10metros
ds_ERA5['vv10m'] = np.sqrt(ds_ERA5['v10']**2 + ds_ERA5['u10']**2)

# Calculando a direção do vento
ds_ERA5['dv10m'] = (np.rad2deg(np.arctan2(-ds_ERA5['u10'], -ds_ERA5['v10'])) + 360) % 360

# Conversão da temperatura do ar de Kelvin para Celsius
ds_ERA5['temp_c'] = ds_ERA5['t2m'] - 273.15

# Conversão da temperatura do ponto de orvalho de Kelvin para Celsius
ds_ERA5['temp_orvalho_c'] = ds_ERA5['d2m'] - 273.15

# Conversão da pressão atmosférica de Pa para hPa
ds_ERA5['pressao_atm'] = ds_ERA5['sp']/100

# Calculando a pressão de vapor de saturação e pressão de vapor real
T = ds_ERA5['temp_c']
Td = ds_ERA5['temp_orvalho_c']

es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
e = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

# Calcula a Umidade Relativa
ds_ERA5['umidade_rel'] = 100 * (e / es)

# Produção de Mapas

In [ ]:
# Seleciona o mês e ano desejado
ano = 2024
mes = 1

# =====================================
# MERGE - Precipitação acumulada mensal
# =====================================
# Seleciona o dado do mês desejado
data_mes_merge = ds_MERGE.sel(time=f'{ano}-{mes:02d}')

# Extrai os valores das coordenadas de longitude e latitude do dataset ERA5
x_merge = ds_MERGE['lon'].values
y_merge = ds_MERGE['lat'].values
# Cria uma grade 2D (malha) combinando todas as longitudes com todas as latitudes
x_merge, y_merge = np.meshgrid(x_merge, y_merge)

# =====================================
# ERA5 - Variável meteorológica (ex: vento, temperatura)
# =====================================
# Seleciona o dado do mês desejado
data_mes_era5 = ds_ERA5.sel(time=f'{ano}-{mes:02d}')

# Extrai os valores das coordenadas de longitude e latitude do dataset ERA5
x_era5 = ds_ERA5['longitude'].values
y_era5 = ds_ERA5['latitude'].values
# Cria uma grade 2D (malha) combinando todas as longitudes com todas as latitudes
x_era5, y_era5 = np.meshgrid(x_era5, y_era5)

# Projeção (exemplo com PlateCarree)
proj = ccrs.PlateCarree()

In [ ]:
# =====================================
# MERGE - Variável meteorológica precipitação
# =====================================
# Seleciona a variável de precipitação acumulada e remove a dimensão 'time'
prcpvar = data_mes_merge['pacum'].squeeze()
data_merge = prcpvar.values  # Dados de precipitação
# =====================================

# Cria uma figura com tamanho 8x8 polegadas
fig = plt.figure(figsize=(8, 8))

# Adiciona um único subplot com projeção geográfica definida em 'proj'
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Desenha os contornos do mapa (litoral, fronteiras e estados)
ax.coastlines()                     # Linhas costeiras
ax.add_feature(cfeature.BORDERS)   # Fronteiras entre países
ax.add_feature(cfeature.STATES)    # Limites dos estados (Brasil)

# Define os níveis de contorno para a precipitação (em mm)
clevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40,
         50, 70, 100, 150, 200, 250, 300, 400, 500, 600, 750]

# Define a paleta de cores personalizada para representar os níveis de chuva
cmap_data = [(1.0, 1.0, 1.0),
             (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
             (0.0, 1.0, 1.0),
             (0.0, 0.8784313797950745, 0.501960813999176),
             (0.0, 0.7529411911964417, 0.0),
             (0.501960813999176, 0.8784313797950745, 0.0),
             (1.0, 1.0, 0.0),
             (1.0, 0.6274510025978088, 0.0),
             (1.0, 0.0, 0.0),
             (1.0, 0.125490203499794, 0.501960813999176),
             (0.9411764740943909, 0.250980406999588, 1.0),
             (0.501960813999176, 0.125490203499794, 1.0),
             (0.250980406999588, 0.250980406999588, 1.0),
             (0.125490203499794, 0.125490203499794, 0.501960813999176),
             (0.125490203499794, 0.125490203499794, 0.125490203499794),
             (0.501960813999176, 0.501960813999176, 0.501960813999176),
             (0.8784313797950745, 0.8784313797950745, 0.8784313797950745),
             (0.9333333373069763, 0.8313725590705872, 0.7372549176216125),
             (0.8549019694328308, 0.6509804129600525, 0.47058823704719543),
             (0.6274510025978088, 0.42352941632270813, 0.23529411852359772),
             (0.4000000059604645, 0.20000000298023224, 0.0)]

# Cria o colormap a partir da paleta definida
cmap = mcolors.ListedColormap(cmap_data, 'precipitation')

# Normaliza os dados com base nos intervalos definidos em clevs
norm = mcolors.BoundaryNorm(clevs, cmap.N)

# Plota os dados de precipitação usando contornos preenchidos
cs = ax.contourf(x_merge, y_merge, data_merge, clevs, cmap=cmap, norm=norm)

# Adiciona a barra de cores (legenda do mapa) na parte inferior
cbar = plt.colorbar(cs, orientation='horizontal', pad=0.03)

# Define o texto da legenda da barra de cores
cbar.set_label('[mm]', fontsize=14)

# Define o título da figura com o mês e ano selecionados
ax.set_title(f"ACUMULADO DE PRECIPITAÇÃO DO MÊS - {ano}/{mes:02d}", fontsize=14, fontweight='bold', y=1.02)

In [ ]:
# =====================================
# ERA5 - Variável meteorológica velocidade do vento
# =====================================
# Seleciona a variável de velocidade do vento e remove a dimensão 'time'
var_era5 = data_mes_era5['vv10m'].squeeze()
data_era5 = var_era5.values
# =====================================

# Cria uma figura com tamanho 8x8 polegadas
fig = plt.figure(figsize=(8, 8))

# Adiciona um único subplot com projeção geográfica definida em 'proj'
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Desenha os contornos do mapa (litoral, fronteiras e estados)
ax.coastlines()                     # Linhas costeiras
ax.add_feature(cfeature.BORDERS)   # Fronteiras entre países
ax.add_feature(cfeature.STATES)    # Limites dos estados (Brasil)

# Define os níveis de contorno para a velocidade do vento (em m/s)
clevs = list(range(0, 11))  # [0, 1, 2, ..., 8]

# Define a paleta de cores personalizada para representar os níveis de chuva
cmap = mpl.cm.jet

# Normaliza os dados com base nos intervalos definidos em clevs
norm = mcolors.BoundaryNorm(clevs, cmap.N)

# Plota os dados de precipitação usando contornos preenchidos
cs = ax.contourf(x_era5, y_era5, data_era5, clevs, cmap=cmap, norm=norm)

# Adiciona a barra de cores (legenda do mapa) na parte inferior
cbar = plt.colorbar(cs, orientation='horizontal', pad=0.03)

# Define o texto da legenda da barra de cores
cbar.set_label('[m/s]', fontsize=14)

# Define o título da figura com o mês e ano selecionados
ax.set_title(f"VELOCIDADE DO VENTO MÉDIA DO MÊS - {mes:02d}/{ano}", fontsize=14, fontweight='bold', y=1.02)

In [ ]:
# =====================================
# ERA5 - Variável meteorológica temperatura do ar
# =====================================
# Seleciona a variável de temperatura do ar e remove a dimensão 'time'
var_era5 = data_mes_era5['temp_c'].squeeze()
data_era5 = var_era5.values
# =====================================

# Cria uma figura com tamanho 8x8 polegadas
fig = plt.figure(figsize=(8, 8))

# Adiciona um único subplot com projeção geográfica definida em 'proj'
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Desenha os contornos do mapa (litoral, fronteiras e estados)
ax.coastlines()                     # Linhas costeiras
ax.add_feature(cfeature.BORDERS)   # Fronteiras entre países
ax.add_feature(cfeature.STATES)    # Limites dos estados (Brasil)

# Define os níveis de contorno para a temperatura do ar (em celsius)
clevs = list(range(0, 41))

# Define a paleta de cores personalizada para representar os níveis de chuva
cmap = mpl.cm.jet

# Normaliza os dados com base nos intervalos definidos em clevs
norm = mcolors.BoundaryNorm(clevs, cmap.N)

# Plota os dados de precipitação usando contornos preenchidos
cs = ax.contourf(x_era5, y_era5, data_era5, clevs, cmap=cmap, norm=norm)

# Adiciona a barra de cores (legenda do mapa) na parte inferior
cbar = plt.colorbar(cs, orientation='horizontal', pad=0.03)

# Define o texto da legenda da barra de cores
cbar.set_label('[°C]', fontsize=14)

# Define o título da figura com o mês e ano selecionados
ax.set_title(f"TEMPERATURA MÉDIA DO MÊS - {mes:02d}/{ano}", fontsize=14, fontweight='bold', y=1.02)